### General goal: To investigate the effects of updating the config on the QM's output. 

#### Specific goal: Can I dynamically update any parameter in the config, without opening a new QM with an updated config?

In [1]:
# ensure relative imports are correct
import sys
sys.path.insert(0, '../..') # this notebook resides in a sub-sub-directory in the main package

In [2]:
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.QuantumMachine import QuantumMachine
from qm.qua import *

from test_qmm_qm_config1 import config1

##### Please restart your Jupyter notebook kernel to purge exisiting variables before beginning each attempt below. This is simply a precaution to ensure that namespace collisions do not cause unnecessary problems...

### Attempt 1. Naively update a value in config1 and hope that the QM follows the update

I don't expect this to work but I'll document it anyway...

In [3]:
qmm = QuantumMachinesManager()

2021-04-13 02:10:53,827 - qm - INFO - Performing health check
2021-04-13 02:10:53,834 - qm - INFO - Health check passed


In [4]:
qm = qmm.open_qm(config1)
print(qm.id)

qm-1618305052926


In [9]:
# play an infinitely long continuous pulse at qubit's IF
with program() as infinite_cw:
    with infinite_loop_():
        play('CW', 'qubit')
job = qm.execute(infinite_cw)
# on the oscilloscope, we see that the QM is playing a pulse at 50MHz

2021-04-13 02:15:49,740 - qm - INFO - Flags: 
2021-04-13 02:15:49,740 - qm - INFO - Executing high level program


In [6]:
# let's try to update the intermediate frequency in a naive way
config1['elements']['qubit']['intermediate_frequency'] = -40e6
print(config1['elements']['qubit']['intermediate_frequency'])

-40000000.0


In [7]:
# restart the job, hope that the QM will play at 40MHz
job = qm.execute(infinite_cw)

2021-04-13 02:02:39,489 - qm - INFO - Flags: 
2021-04-13 02:02:39,489 - qm - INFO - Executing high level program


#### Result: Unsuccessful, as expected. To succeed, we would need to open a new qm with the updated config, which defeats the purpose of the Specific goal outlined in the first cell of this notebook.

#### The correct way to update intermediate frequency, as of version 0.8, is to use the set_intermediate_frequency method of the QM API

In [17]:
# running this immediately changes the int_freq
qm.set_intermediate_frequency('qubit', -50e6)
# you may run the following command to verify that the qm config has been updated too
#qm.get_config()

#### oscillator frequency can only be updated within the qua program context manager, and this update does not change the qm config. 

In [15]:
with program() as freq_update:
    update_frequency("qubit", -40e6)
qm.execute(freq_update)
#qm.get_config() # if you run this you notice that the lo_freq of the element is not updated in the qm config

2021-04-13 02:09:28,844 - qm - INFO - Flags: 
2021-04-13 02:09:28,845 - qm - INFO - Executing high level program


#### but update_frequency does affect the freq of the pulse played by qm. let's find out by playing a long CW and waiting a long time between iterations. let's also set the start, stop, and step such that we can observe the output changing on an oscilloscope (make the time scale to the order of 100 nanoseconds).

In [34]:
f_start = 0.01e6
f_stop = 100e6
f_step = 1e6
wait_time = int(1e7)

with program() as freq_update:
    freq = declare(int)

    with for_(freq, f_start, freq < f_stop, freq + f_step):
        update_frequency('qubit', freq)
        play('CW', 'qubit', duration=int(1e7))
        wait(wait_time, 'qubit')

job = qm.execute(freq_update)

2021-04-13 02:32:05,957 - qm - INFO - Flags: 
2021-04-13 02:32:05,958 - qm - INFO - Executing high level program
